# 神经网络训练和推理流程概览

本Notebook使用机器学习入门经典数据集Iris来演示神经网络的训练和推理（测试）流程。

Iris是鸢尾花分类数据集，其中每一条数据包含花萼长度、花萼宽度、花瓣长度、花瓣宽度、以及花的品种，如下所示：

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()  # 加载iris数据集
X_data = pd.DataFrame(iris.data, columns=iris.feature_names)  # iris.data是输入
y_data = pd.DataFrame(iris.target, columns=['target'])  # iris.target是标签
data = pd.concat([X_data, y_data], axis=1)
data_train, data_test = train_test_split(data, test_size=0.1, random_state=42)  # 划分训练集和测试集

In [ ]:
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


我们将花萼长度、花萼宽度、花瓣长度、花瓣宽度分别记为$x_1,x_2,x_3,x_4$，花的品种记为$y$

我们的目标是找到一个函数$f$，可以输入$x_1,x_2,x_3,x_4$，输出花的品种$y$.

我们可以将函数$f$构造为最简单的线性函数$f(x_1,x_2,x_3,x_4)=w_1x_1+w_2x_2+w_3x_3+w_4x_4+b$，其中$w_1,w_2,w_3,w_4$都是需要学习的参数：


## 1. 直观的函数定义

In [ ]:
import torch
from torch import nn


class F1(nn.Module):

  def __init__(self):
    super().__init__()
    # 将函数的参数定义在__init__方法中
    self.w1 = nn.Parameter(torch.tensor(0.1))
    self.w2 = nn.Parameter(torch.tensor(0.2))
    self.w3 = nn.Parameter(torch.tensor(0.3))
    self.w4 = nn.Parameter(torch.tensor(0.4))
    self.b = nn.Parameter(torch.tensor(0.0))

  def forward(self, x1, x2, x3, x4):
    # 将函数的调用过程定义在forward方法中
    return (
      self.w1*x1+
      self.w2*x2+
      self.w3*x3+
      self.w4*x4+
      self.b
    )


# 初始化函数
f1 = F1()
# 调用函数（把第一条数据代入函数计算）
y_pred = f1(torch.tensor(5.1), torch.tensor(3.5), torch.tensor(1.4), torch.tensor(0.2))
print(y_pred)

tensor(1.7100, grad_fn=<AddBackward0>)


可以看到，在上面的代码中，我们将$f$的参数$w_1,w_2,w_3,w_4,b$分别初始化为$0.1,0.2,0.3,0.4,0.0$（也可以随机初始化），将第一组数据$(5.1,3.5,1.4,0.2)$代入$f$计算得到$f(5.1,3.5,1.4,0.2)=1.71$。

## 2. 向量化

上面的代码太过冗余，5个参数定义了5次，并且调用时需要传入4个tensor，我们可以把参数和函数的输入向量化：

In [ ]:
import torch
from torch import nn


class F2(nn.Module):

  def __init__(self):
    super().__init__()
    self.w = nn.Parameter(torch.tensor([0.1,0.2,0.3,0.4]))
    self.b = nn.Parameter(torch.tensor(0.0))

  def forward(self, x):
    # 用向量(w1,w2,w3,w4)点乘向量(x1,x2,x3,x4)，再加上b，与原始表达式等价
    return self.w.dot(x) + self.b


# 初始化函数
f2 = F2()
# 调用函数（把第一条数据代入函数计算）
y = f2(torch.tensor([5.1, 3.5, 1.4, 0.2]))
print(y)

tensor(1.7100, grad_fn=<AddBackward0>)


向量化后，$w_1,w_2,w_3,w_4$四个参数被定义在了一个向量中，$x_1,x_2,x_3,x_4$四个输入也被包含在了一个向量中，代码简洁了很多。

## 3. 批量化

上述函数每次只允许我们输入一条数据，没有办法攒多条数据一起丢给函数让它并行地算出结果来。

在上述函数中，我们已经把参数$w$看作了一个向量：$\mathbf{w}=\left[\begin{aligned}w_1\\w_2\\w_3\\w_4\end{aligned}\right]$

也把每条数据看成了一个向量：$\mathbf{x}^{(i)}=\left[\begin{aligned}x_1^{(i)}\\x_2^{(i)}\\x_3^{(i)}\\x_4^{(i)}\end{aligned}\right]$，其中上标$(i)$表示第$i$条数据

计算过程也变为了参数向量与数据向量的点乘：$f(\mathbf{x}^{(i)})=\mathbf{w}^T\mathbf{x}^{(i)}=w_1x_1^{(i)}+w_2x_2^{(i)}+w_3x_3^{(i)}+w_4x_4^{(i)}$

现在，我们把3条数据攒成一个batch，并看作一个矩阵：$
X^{(i:i+2)}=\left[
\begin{aligned}
x_1^{(i)}~x_1^{(i+1)}~x_1^{(i+2)}\\
x_2^{(i)}~x_2^{(i+1)}~x_2^{(i+2)}\\
x_3^{(i)}~x_3^{(i+1)}~x_3^{(i+2)}\\
x_4^{(i)}~x_4^{(i+1)}~x_4^{(i+2)}
\end{aligned}\right]
$

此时，仍然使用参数向量的转置$\mathbf{w}^T$乘上这个矩阵，我们就能得到一个向量，这个向量包含每条数据的函数值：

$$
\mathbf{w}^TX^{(i:i+2)}=[w_1,w_2,w_3,w_4]\cdot\left[
\begin{aligned}
x_1^{(i)}~x_1^{(i+1)}~x_1^{(i+2)}\\
x_2^{(i)}~x_2^{(i+1)}~x_2^{(i+2)}\\
x_3^{(i)}~x_3^{(i+1)}~x_3^{(i+2)}\\
x_4^{(i)}~x_4^{(i+1)}~x_4^{(i+2)}
\end{aligned}\right]=[f(\mathbf{x}^{(i)}),f(\mathbf{x}^{(i+1)}),f(\mathbf{x}^{(i+2)})]
$$

如此一来，3条数据的函数值通过矩阵乘法被并行地计算了出来。

代码实现如下：

In [ ]:
import torch
from torch import nn


class F3(nn.Module):

  def __init__(self):
    super().__init__()
    self.w = nn.Parameter(torch.tensor([[0.1,0.2,0.3,0.4]]))  # 多加一层括号就从4维向量变成了1x4的矩阵，然后就可以跟矩阵相乘了
    self.b = nn.Parameter(torch.tensor(0.0))

  def forward(self, x):
    return self.w @ x.T + self.b  # 这里的实现是w乘上x的转置，和公式里相反，因为代码里w是行向量，x中的每个xi也是行向量


# 初始化函数
f3 = F3()
# 调用函数（把前2条数据攒成batch代入函数计算）
y = f3(
  torch.tensor([
    [5.1, 3.5, 1.4, 0.2],
    [4.9, 3.0, 1.4, 0.2],
  ])
)
print(y)

tensor([1.7100, 1.5900], grad_fn=<AddBackward0>)


## 4. 计算损失

我们需要定义一个损失函数来评量函数$f$的好坏，损失函数越小说明函数$f$越好。

我们将损失函数定义为函数$f$在整个数据集$\mathcal{D}$上的预测值与真实值的均方误差：

$$
L(f)=\frac{1}{|\mathcal{D}|}\sum_{(\mathbf{x},y)\in\mathcal{D}}(f(\mathbf{x})-y)^2
$$

代码实现为：

In [ ]:
def loss_function(y_pred, y):
  return ((y_pred - y)**2).mean()

我们需要遍历每一条数据$\mathbf{x}^{(i)},y^{(i)}$，通过函数$f$计算预测值$f(\mathbf{x}^{(i)})$，并计算预测值与真实值$y^{(i)}$之间的均方误差，最后加起来可以得到损失函数的值。

为了加快速度，我们将`batch_size`设为4，攒4条数据一起丢给函数进行预测：

In [ ]:
import numpy as np


f = F3()
loss_list = []
for i in range(0, len(data_train), 4):  # 只在训练集上计算
  x = torch.tensor(np.array(data_train.iloc[i:i+4, :-1]), dtype=torch.float32)
  y = torch.tensor(np.array(data_train.iloc[i:i+4, -1]), dtype=torch.float32)
  y_pred = f(x)
  loss = loss_function(y_pred, y)
  loss_list.append(loss)
total_loss = sum(loss_list)
print("Total loss:", total_loss)

Total loss: tensor(112.3935, grad_fn=<AddBackward0>)


## 5. 反向传播

上面我们计算出了函数$f$在当前的参数下，在整个Iris数据集上的损失。

接下来我们要使用AdamW优化器对函数$f$的参数进行更新，使损失可以减小一点：

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(
    params=f.parameters(),  # 需要优化的参数
    lr=0.1,  # 学习率，即参数更新的步伐大小
)

可以看到，优化器中已经载入了$f$中定义的参数$\mathbf{w}=[0.1,0.2,0.3,0.4]^T$以及$b=0.0$：

In [ ]:
optimizer.param_groups[0]["params"]

[Parameter containing:
 tensor([[0.1000, 0.2000, 0.3000, 0.4000]], requires_grad=True),
 Parameter containing:
 tensor(0., requires_grad=True)]

接下来，我们调用`total_loss`的`backward`方法。这一做法会让pytorch在后端计算出`total_loss`关于参数`w`和参数`b`的梯度，即$\nabla_{\mathbf{w}}L(f)=\frac{\partial L(f)}{\partial \mathbf{w}}$和$\nabla_{b}L(f)=\frac{\partial L(f)}{\partial b}$。

In [ ]:
total_loss.backward()

$L$关于$\mathbf{w}$的梯度$\nabla_{\mathbf{w}}L(f)$是一个形状与参数$\mathbf{w}$相同的向量，它表示一个方向，当$\mathbf{w}$沿此方向改变时，对$L$的改变最大。

$\nabla_{b}L(f)$同理。

通过调用`optimizer.step()`可以使`optimizer`中载入的参数沿梯度所指的方向更新参数，从而使$L$变小：

In [ ]:
optimizer.step()

我们可以看看，函数$f$的参数现在变成什么样子了：

In [ ]:
print(f.w)
print(f.b)

Parameter containing:
tensor([[-1.0000e-04,  9.9800e-02,  1.9970e-01,  2.9960e-01]],
       requires_grad=True)
Parameter containing:
tensor(-0.1000, requires_grad=True)


我们可以看到，`w`中每个值都减小了$0.01$，`b`也减小了0.01，这可以反推出`total_loss.backward()`中计算出的梯度为$\nabla_{\mathbf{w}}L(f)\approx[0.01,0.01,0.01,0.01]^T$，以及$\nabla_{\mathbf{w}}L(f)\approx 0.01$

再次计算loss可以发现，loss比之前确实减小了：

In [ ]:
loss_list = []
for i in range(0, len(data_train), 4):
  x = torch.tensor(np.array(data_train.iloc[i:i+4, :-1]), dtype=torch.float32)
  y = torch.tensor(np.array(data_train.iloc[i:i+4, -1]), dtype=torch.float32)
  y_pred = f(x)
  loss = loss_function(y_pred, y)
  loss_list.append(loss)
total_loss = sum(loss_list)
print("Total loss:", total_loss)

Total loss: tensor(6.8916, grad_fn=<AddBackward0>)


重复上述过程，对数据集遍历100个`epoch`，并观察loss如何变化：

In [ ]:
num_epochs = 100
for i_epoch in range(num_epochs):
  # 前向传播
  loss_list = []
  for i in range(0, len(data_train), 4):
    x = torch.tensor(np.array(data_train.iloc[i:i+4, :-1]), dtype=torch.float32)
    y = torch.tensor(np.array(data_train.iloc[i:i+4, -1]), dtype=torch.float32)
    y_pred = f(x)
    loss = loss_function(y_pred, y)
    loss_list.append(loss)
  total_loss = sum(loss_list)
  print("Total loss:", total_loss.item())

  # 反向传播
  optimizer.zero_grad()  # 每次backward之前要先清空上次累积在后台的梯度
  total_loss.backward()
  optimizer.step()

Total loss: 6.891600608825684
Total loss: 33.027713775634766
Total loss: 59.78230667114258
Total loss: 41.90623092651367
Total loss: 13.976655006408691
Total loss: 3.4685986042022705
Total loss: 13.15591049194336
Total loss: 26.092050552368164
Total loss: 27.26593780517578
Total loss: 17.30009651184082
Total loss: 6.126542568206787
Total loss: 2.2355329990386963
Total loss: 6.526439189910889
Total loss: 12.894046783447266
Total loss: 14.734004974365234
Total loss: 10.780180931091309
Total loss: 4.886590480804443
Total loss: 1.774026870727539
Total loss: 3.096604585647583
Total loss: 6.549545764923096
Total loss: 8.431142807006836
Total loss: 7.095152854919434
Total loss: 3.9898970127105713
Total loss: 1.8500605821609497
Total loss: 2.1487560272216797
Total loss: 4.006176471710205
Total loss: 5.311361789703369
Total loss: 4.818202972412109
Total loss: 3.1321418285369873
Total loss: 1.8517488241195679
Total loss: 1.9548126459121704
Total loss: 2.9883344173431396
Total loss: 3.70512557029

观察到，loss降到1.64左右时没法再下降了。

这说明，在当前的超参数配置下，函数/模型$f$在Iris数据集上的训练已经收敛。

我们看看收敛后的参数：

In [ ]:
print(f.w)
print(f.b)

Parameter containing:
tensor([[-0.0547, -0.0366,  0.2676,  0.4737]], requires_grad=True)
Parameter containing:
tensor(-0.1373, requires_grad=True)


## 6. 模型测试

接下来我们对训练好的模型$f$在测试集上进行测试，评估$f$在测试集上的loss:

In [ ]:
# 前向传播
loss_list = []
y_preds = []
y_trues = []
for i in range(0, len(data_test), 4):
  x = torch.tensor(np.array(data_test.iloc[i:i+4, :-1]), dtype=torch.float32)
  y = torch.tensor(np.array(data_test.iloc[i:i+4, -1]), dtype=torch.float32)
  y_pred = f(x)
  y_preds.extend(y_pred.squeeze().tolist())
  y_trues.extend(y.squeeze().tolist())
  loss = loss_function(y_pred, y)
  loss_list.append(loss)
avg_loss = sum(loss_list)
print("Average loss:", avg_loss)

Average loss: tensor(0.1654, grad_fn=<AddBackward0>)


另外还需计算$f$在测试集上的准确率。这里我们的预测值是连续值，但是真实值，我们直接取其四舍五入的整数值作为最终的分类预测：

In [ ]:
y_preds = [round(y_pred) for y_pred in y_preds]

In [ ]:
y_preds

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0]

准确率轻松地达到了100%：

In [ ]:
accuracy = (np.array(y_preds) == np.array(y_trues)).mean()
print("Accuracy:", accuracy)

Accuracy: 1.0


In [ ]:
np.array(y_preds) == np.array(y_trues)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

## 7. 让函数输出概率分布

我们上面定义的函数$f$输入是鸢尾花的各项特征$\mathbf{x}$，输出是一个连续的数值，代表花的品种。

我们评量$f$的好坏时，计算的是预测数值$f(\mathbf{x})$与真实品种$\mathbf{y}$之间的均方误差。

然而，这种做法对不同的类别而言不公平。

例如，当预测类别$f(\mathbf{x})=0$时，如果真实类别为$\mathbf{y}=1$，则损失为$1$，但如果真实类别为$2$，则损失为2。

同样是类别预测错误，真实类别为$2$时被认定为犯了更严重的错误，这对于类别$1$来说不公平。

因此，在分类问题中，我们通常不直接输出代表类别的值，而是为每一个类别输出一个概率，形成一个**多分类概率分布（Categorical Distribution）**。

因此，对于一条输入数据$\mathbf{x}$，我们的函数定义应该变为：

$$
\left[\begin{aligned}
y_1 \\
y_2 \\
y_3
\end{aligned}
\right] = \mathbf{f}(\mathbf{x})=W^T\mathbf{x}+\mathbf{b}=\left[\begin{aligned}
w_{11}~w_{12}~w_{13}~w_{14} \\
w_{21}~w_{22}~w_{13}~w_{24} \\
w_{31}~w_{32}~w_{13}~w_{34}
\end{aligned}\right]\cdot\left[\begin{aligned}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{aligned}
\right]+\left[\begin{aligned}
b_1 \\
b_2 \\
b_3
\end{aligned}
\right]
$$

我们希望$y_1,y_2,y_3$分别代表花属于3种品种的概率，因此需要保证它们之和为$1$，因此我们再加入softmax函数：

$$
\mathbf{f}(\mathbf{x})=\text{softmax}(W^T\mathbf{x}+\mathbf{b})
$$

这一函数的代码实现如下：

In [ ]:
import torch
from torch import nn


class F4(nn.Module):

  def __init__(self):
    super().__init__()
    self.W = nn.Parameter(torch.tensor([
      [0.1,0.2,0.3,0.4],
      [0.2,0.3,0.4,0.1],
      [0.3,0.4,0.1,0.2],
    ]))
    self.b = nn.Parameter(torch.tensor([0.0,0.1,0.2]))
    self.softmax = nn.Softmax(-1)

  def forward(self, x):
    return self.softmax((self.W @ x.T).T + self.b)


# 初始化函数
f4 = F4()
# 调用函数（把前2条数据攒成batch代入函数计算）
y_dist = f4(
  torch.tensor([
    [5.1, 3.5, 1.4, 0.2],
    [4.9, 3.0, 1.4, 0.2],
  ])
)
print(y_dist)

tensor([[0.1139, 0.3222, 0.5640],
        [0.1259, 0.3321, 0.5420]], grad_fn=<SoftmaxBackward0>)


需要注意，代码实现与公式的描述有些差异，但是是等价的：

- 代码中的`self.W`其实是公式中的$W^T$，因此不再需要对`self.w`进行转置
- 代码中的`x`是行向量，公式中的$\mathbf{x}$是列向量，因此将`x`转置为`x.T`
- 公式中的$W^T\mathbf{x}$即为代码中的`self.w @ x.T`
- 代码中的`b`是行向量，公式中的$\mathbf{b}$是列向量，按道理应该对`b`进行转置，再与$W^T\mathbf{x}$相加；但是这里`b`是一维的tensor（一维数组），无法转置，因此我们直接对`self.w @ x.T`再做一次转置成为`(self.w @ x.T).T`，再与`b`相加。
- 代码中的输出是行向量，公式中的输出是列向量

可以看到，我们给函数传入了一个大小为2的batch，其输出如下：

- 第1条数据：11.39%概率为第0类品种，32.22%概率为第1类品种，56.40%概率为第2类品种
- 第2条数据：12.59%概率为第0类品种，33.21%概率为第1类品种，54.20%概率为第2类品种

这里，我们用到了一个$3\times4$的矩阵作为参数$W^T$，以及一个3维的向量作为$b$，并把它们的初始值都硬编码在了`__init__`中。

实际上，我们可以让它的初始值为随机值：

In [ ]:
import torch
from torch import nn

torch.random.manual_seed(42)  # 固定随机种子以便复现

class F5(nn.Module):

  def __init__(self):
    super().__init__()
    self.W = nn.Parameter(torch.randn(3, 4))  # 3x4的矩阵
    self.b = nn.Parameter(torch.rand(3))  # 3维的向量
    self.softmax = nn.Softmax(-1)

  def forward(self, x):
    return self.softmax((self.W @ x.T).T + self.b)


# 初始化函数
f5 = F5()
# 调用函数（把前2条数据攒成batch代入函数计算）
y_dist = f5(
  torch.tensor([
    [5.1, 3.5, 1.4, 0.2],
    [4.9, 3.0, 1.4, 0.2],
  ])
)
print(y_dist)

tensor([[1.2869e-01, 3.9238e-04, 8.7092e-01],
        [1.3959e-01, 6.6718e-04, 8.5974e-01]], grad_fn=<SoftmaxBackward0>)


这里，`W`和`b`的目的是构建一个形如$\mathbf{f}(X)=W^TX+b$的线性函数，该线性函数可以将4维的输入数据$[x_1,x_2,x_3,x_4]$映射为三维的品种概率分布$[y_1,y_2,y_3]$.

在pytorch中有一个更简单的模块可以实现这一功能，即`nn.Linear`:

In [ ]:
import torch
from torch import nn

torch.random.manual_seed(42)  # 固定随机种子以便复现

class F6(nn.Module):

  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3) # 输入为4维，输出为3维
    self.softmax = nn.Softmax(-1)

  def forward(self, x):
    return self.softmax(self.linear(x))


# 初始化函数
f6 = F6()
# 调用函数（把前3条数据攒成batch代入函数计算）
X = torch.tensor([
  [5.1, 3.5, 1.4, 0.2],
  [4.9, 3.0, 1.4, 0.2],
])
y_dist = f6(X)
print(y_dist)

tensor([[0.8541, 0.0139, 0.1320],
        [0.8020, 0.0168, 0.1811]], grad_fn=<SoftmaxBackward0>)


`nn.Linear`模块中包含了参数W和b，如下：

In [ ]:
print(f6.linear.weight)  # W
print(f6.linear.bias)  # b

Parameter containing:
tensor([[ 0.3823,  0.4150, -0.1171,  0.4593],
        [-0.1096,  0.1009, -0.2434,  0.2936],
        [ 0.4408, -0.3668,  0.4346,  0.0936]], requires_grad=True)
Parameter containing:
tensor([0.3694, 0.0677, 0.2411], requires_grad=True)


将数据`X`直接丢给`nn.Linear`等价于进行`WX+b`的运算:

In [ ]:
f6.linear(X)

tensor([[ 3.6994, -0.4199,  1.8323],
        [ 3.4154, -0.4485,  1.9276]], grad_fn=<AddmmBackward0>)

In [ ]:
(f6.linear.weight @ X.T).T + f6.linear.bias

tensor([[ 3.6994, -0.4199,  1.8323],
        [ 3.4154, -0.4485,  1.9276]], grad_fn=<AddBackward0>)

## 8. 最大化真实标签的概率

上述函数输出了长度为3的向量，其中每个值表示鸢尾花属于每个类别的概率。

我们需要最大化真实类别的概率。

假设模型对于一个batch中4条输入所输出的概率分布为：


In [ ]:
mock_y_dist = torch.tensor([
    [0.1, 0.2, 0.7],
    [0.8, 0.1, 0.1],
    [0.6, 0.2, 0.2],
    [0.2, 0.7, 0.1]
])

假设4条输入对应的真实标签为：

In [ ]:
mock_y_true = torch.tensor([2, 0, 2, 1])

我们可以批量地使用真实标签作为索引，取出真实标签的预测概率：

In [ ]:
mock_probs = mock_y_dist.gather(dim=1, index=mock_y_true[..., None])

这里，`gather`操作用`mock_y_true`中的每个整数作为索引，提取出`mock_y_dist`中每一行对应位置的值：

In [ ]:
mock_probs

tensor([[0.7000],
        [0.8000],
        [0.2000],
        [0.7000]])

我们将损失定义为所有`probs`的负对数的平均值：

In [ ]:
mock_loss = -mock_probs.log().mean()

In [ ]:
mock_loss

tensor(0.6365)

损失越小，代表真实标签概率的负对数越小，代表真实标签的概率越大，从而代表模型对于真实标签预测的质量越好。

将上述mock的过程定义为一个批量计算分类损失的函数：

In [ ]:
def cross_entropy_loss(y_dist, y_true):
  probs = y_dist.gather(dim=1, index=y_true[..., None])
  return -probs.log().mean()

接下来开始训练：

In [ ]:
f = F6()
optimizer = AdamW(f.parameters(), lr=0.1)
num_epochs = 100
for i_epoch in range(num_epochs):
  # 前向传播
  loss_list = []
  for i in range(0, len(data_train), 4):
    x = torch.tensor(np.array(data_train.iloc[i:i+4, :-1]), dtype=torch.float32)
    y = torch.tensor(np.array(data_train.iloc[i:i+4, -1]), dtype=torch.long)
    y_dist = f(x)
    loss = cross_entropy_loss(y_dist, y)
    loss_list.append(loss)
  total_loss = sum(loss_list)
  print("Total loss:", total_loss.item())

  # 反向传播
  optimizer.zero_grad()  # 每次backward之前要先清空上次累积在后台的梯度
  total_loss.backward()
  optimizer.step()

Total loss: 42.622169494628906
Total loss: 40.01057815551758
Total loss: 45.771907806396484
Total loss: 39.62301254272461
Total loss: 30.42124366760254
Total loss: 27.673301696777344
Total loss: 26.51082992553711
Total loss: 26.32590675354004
Total loss: 25.836532592773438
Total loss: 23.69374656677246
Total loss: 21.142059326171875
Total loss: 19.087554931640625
Total loss: 18.144611358642578
Total loss: 18.369930267333984
Total loss: 18.445894241333008
Total loss: 17.724994659423828
Total loss: 17.006301879882812
Total loss: 16.62527084350586
Total loss: 16.06058692932129
Total loss: 15.18472957611084
Total loss: 14.506752014160156
Total loss: 14.268475532531738
Total loss: 14.059664726257324
Total loss: 13.662631034851074
Total loss: 13.374229431152344
Total loss: 13.30797290802002
Total loss: 13.185258865356445
Total loss: 12.869413375854492
Total loss: 12.551698684692383
Total loss: 12.360967636108398
Total loss: 12.149430274963379
Total loss: 11.829717636108398
Total loss: 11.551

测试模型性能：

In [ ]:
# 前向传播
loss_list = []
y_preds = []
y_trues = []
for i in range(0, len(data_test), 4):
  x = torch.tensor(np.array(data_test.iloc[i:i+4, :-1]), dtype=torch.float32)
  y = torch.tensor(np.array(data_test.iloc[i:i+4, -1]), dtype=torch.long)
  y_dist = f(x)
  y_pred = y_dist.argmax(dim=1)  # 取概率最大的类别作为预测值
  y_preds.extend(y_pred.tolist())
  y_trues.extend(y.tolist())
  loss = cross_entropy_loss(y_dist, y)
  loss_list.append(loss)
avg_loss = sum(loss_list) / len(loss_list)
print("Average loss:", avg_loss)

Average loss: tensor(0.1600, grad_fn=<DivBackward0>)


In [ ]:
accuracy = (np.array(y_preds) == np.array(y_trues)).mean()
print("Accuracy:", accuracy)

Accuracy: 1.0
